<a href="https://colab.research.google.com/github/navymaddox/Ship_Web_Scraping/blob/main/WarshipWebScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

!pip install selenium #needed for web scrapers
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver #needed to we scrape from Google Chrome
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

#the below chrome_options are needed in order to run this on Google Colab
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.webite-url.com")

!pip install pillow
import PIL
import time
import requests
import os
from PIL import Image
import io
import hashlib
import shutil

     |████████████████████████████████| 954 kB 5.5 MB/s 
     |████████████████████████████████| 138 kB 67.1 MB/s 
     |████████████████████████████████| 356 kB 67.4 MB/s 
     |████████████████████████████████| 3.5 MB 65.0 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Practicum/Ships/List of Ships.xlsx', sheet_name= None)
df

{'IRIN':            Class  Total Number  ...     Type           Origin
 0   Alvand Class           3.0  ...  Frigate   United Kingdom
 1            NaN           NaN  ...      NaN              NaN
 2            NaN           NaN  ...      NaN              NaN
 3            NaN           NaN  ...      NaN              NaN
 4            NaN           NaN  ...      NaN              NaN
 ..           ...           ...  ...      ...              ...
 80           NaN           NaN  ...      NaN              NaN
 81           NaN           NaN  ...      NaN              NaN
 82           NaN           NaN  ...      NaN              NaN
 83           NaN           NaN  ...      NaN              NaN
 84           NaN           NaN  ...      NaN              NaN
 
 [85 rows x 8 columns],
 'PRC':                     Class  Total Number  ...            Fleet              Status
 0    Kuznetsov Class Ship           4.0  ...  North Sea Fleet              Active
 1                     NaN           

In [ ]:
PRC = df['PRC'].Class
PRC = PRC.dropna()
PRCList = PRC.to_list()
PRCList

In [ ]:
IRIN = df['IRIN'].Class
IRIN = IRIN.dropna()
IRINlist = IRIN.tolist()
IRINlist
IRINlist = [ s.replace('\xa0', ' ') for s in IRINlist if not pd.isna(s) ] #excel bug that adds these metadata bits to the names
IRINlist = [ s.replace('.', '_') for s in IRINlist if not pd.isna(s) ] #in order to add the folders correctly later, the "." has to be replaced with "_"
IRINlist

Time for the Web Scrape!

In [ ]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [ ]:
def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [ ]:
#This creates the IRIN and PRC folders in my Google Drive
os.makedirs('/content/drive/MyDrive/Practicum/Ships/IRIN')
os.makedirs('/content/drive/MyDrive/Practicum/Ships/PRC')

In [ ]:
if __name__ == '__main__':
    queries = ['Alvand Class Ship',
 'Moudge Class Ship',
 'Bayandor Class Ship',
 'Hamzeh Class Ship',
 'Kaman Class Ship',
 'Sina Class Ship',
 'Kaivan Class Ship',
 'Parvin Class Ship',
 'Hengam Class Ship',
 'Karbala Class Ship',
 'SR.N6 Class Ship',
 'BH.7 Class Ship']  #change your set of querries here
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,30,wd)
        images_path = '/content/drive/MyDrive/Practicum/Ships/IRIN'  #enter your desired image path
        for i in links:
            persist_image(images_path,query,i)

In [ ]:
#The Webdriver will disconnect after the first run and must be restarted for the next web scrape
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.webite-url.com")

In [ ]:
if __name__ == '__main__':
    queries = ['Kuznetsov Class Ship',
 'Yushen Class Ship',
 'Yuzhao Class Ship',
 'Yuting III Class Ship',
 'Yuting II Class Ship',
 'Yuting Class Ship',
 'Yunshu Class Ship',
 'Yudeng Class Ship',
 'Yubei Class Ship',
 'Yuhai Class Ship',
 'Renhai Class Ship',
 'Luyang III Class Ship',
 'Luyang II Class Ship',
 'Luzhou Class Ship',
 'Luyang I Class Ship',
 'Sovremenny Class Ship',
 'Luhai Class Ship',
 'Luhu Class Ship',
 'Jiangkai II Class Ship',
 'Jiangkai I Class Ship',
 'Jiangwei II Class Ship',
 'Jianghu Class Ship',
 'Jiangdao Class Ship',
 'Houbei Class Ship',
 'Houjian Class Ship',
 'Houxin Class Ship',
 'Haiqing Class Ship',
 'Shanghai III Class Ship',
 'Wochi Class Ship',
 'Wozang Class Ship',
 'Wosao Class Ship',
 'Fuyu Class Ship']  #change your set of querries here
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,30,wd)
        images_path = '/content/drive/MyDrive/Practicum/Ships/PRC'  #enter your desired image path
        for i in links:
            persist_image(images_path,query,i)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator()
batches_consol   = generator.flow_from_directory('/content/drive/MyDrive/Practicum/Ships')

indices = batches_consol.class_indices

labels  = list(indices.keys())
print(labels)

In [ ]:
#This is handy method to quickly display the images that have been collected and delete from the here
!pip install -q jmd_imagescraper
from jmd_imagescraper.imagecleaner import *

In [ ]:
display_image_cleaner('/content/drive/MyDrive/Practicum/Ships/IRIN')

HTML(value='<h2>No images left to display in this folder.</h2>', layout=Layout(visibility='hidden'))

GridBox(children=(VBox(children=(Image(value=b'', layout="Layout(width='150px')"), Button(description='Delete'…

In [ ]:
display_image_cleaner('/content/drive/MyDrive/Practicum/Ships/PRC')

HTML(value='<h2>No images left to display in this folder.</h2>', layout=Layout(visibility='hidden'))

GridBox(children=(VBox(children=(Image(value=b'', layout="Layout(width='150px')"), Button(description='Delete'…

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator()
batches_consol   = generator.flow_from_directory('/content/drive/MyDrive/Practicum/Ships')

indices = batches_consol.class_indices

labels  = list(indices.keys())
print(labels)

Found 876 images belonging to 47 classes.
['.ipynb_checkpoints', 'Alvand Class Ship', 'BH7 Class Ship', 'Bandar Abbas Class Ship', 'Bayandor Class Ship', 'Delvar Class Ship', 'Fuyu Class Ship', 'Haiqing Class Ship', 'Hamzeh Class Ship', 'Hendijan Class Ship', 'Hengam Class Ship', 'Houbei Class Ship', 'Houjian Class Ship', 'Houxin Class Ship', 'Jiangdao Class Ship', 'Jianghu Class Ship', 'Jiangkai I Class Ship', 'Jiangkai II Class Ship', 'Jiangwei II Class Ship', 'Kaivan Class Ship', 'Kaman Class Ship', 'Kuznetsov Class Ship', 'Luhai Class Ship', 'Luhu Class Ship', 'Luyang I Class Ship', 'Luyang II Class Ship', 'Luyang III Class Ship', 'Luzhou Class Ship', 'Moudge Class Ship', 'Parvin Class Ship', 'Renhai Class Ship', 'SRN6 Class Ship', 'Shanghai III Class Ship', 'Sina Class Ship', 'Sovremenny Class Ship', 'Wochi Class Ship', 'Wosao Class Ship', 'Wozang Class Ship', 'Yubei Class Ship', 'Yudeng Class Ship', 'Yuhai Class Ship', 'Yunshu Class Ship', 'Yushen Class Ship', 'Yuting Class Ship'

In [ ]:
import os
os.getcwd()
collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Alvand Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Alvand Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Alvand Class Ship/" + "IRIN Alvand Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/BH7 Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/BH7 Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/BH7 Class Ship/" + "IRIN BH7 Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Bandar Abbas Class Ship/"
for i, filename in enumerate(os.listdir(collection)):    
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Bandar Abbas Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Bandar Abbas Class Ship/" + "IRIN Bandar Abbas Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Bayandor Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Bayandor Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Bayandor Class Ship/" + "IRIN Bayandor Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Delvar Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Delvar Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Delvar Class Ship/" + "IRIN Delvar Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Hamzeh Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Hamzeh Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Hamzeh Class Ship/" + "IRIN Hamzeh Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Hendijan Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Hendijan Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Hendijan Class Ship/" + "IRIN Hendijan Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Hengam Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Hengam Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Hengam Class Ship/" + "IRIN Hengam Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Kaivan Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Kaivan Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Kaivan Class Ship/" + "IRIN Kaivan Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Kaman Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Kaman Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Kaman Class Ship/" + "IRIN Kaman Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Moudge Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Moudge Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Moudge Class Ship/" + "IRIN Moudge Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Parvin Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Parvin Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Parvin Class Ship/" + "IRIN Parvin Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/SRN6 Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/SRN6 Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/SRN6 Class Ship/" + "IRIN SRN6 Class" + str(i) + ".jpg")
    
collection = "/content/drive/MyDrive/Practicum/Ships/IRIN/Sina Class Ship/"
for i, filename in enumerate(os.listdir(collection)): 
    os.rename("/content/drive/MyDrive/Practicum/Ships/IRIN/Sina Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/IRIN/Sina Class Ship/" + "IRIN Sina Class" + str(i) + ".jpg")

In [ ]:
os.getcwd()
collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Fuyu Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Fuyu Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Fuyu Class Ship/" + "PRC Fuyu Class" + str(i) + ".jpg")
    
collection = "//content/drive/MyDrive/Practicum/Ships/PRC/Haiqing Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("//content/drive/MyDrive/Practicum/Ships/PRC/Haiqing Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Haiqing Class Ship/" + "PRC Haiqing Class" + str(i) + ".jpg")

collection = "//content/drive/MyDrive/Practicum/Ships/PRC/Houbei Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("//content/drive/MyDrive/Practicum/Ships/PRC/Houbei Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Houbei Class Ship/" + "PRC Houbei Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Houjian Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Houjian Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Houjian Class Ship/" + "PRC Houjian Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Houxin Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Houxin Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Houxin Class Ship/" + "PRC Houxin Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Jiangdao Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Jiangdao Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Jiangdao Class Ship/" + "PRC Jiangdao Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Jianghu Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Jianghu Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Jianghu Class Ship/" + "PRC Jianghu Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Jiangkai I Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Jiangkai I Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Jiangkai I Class Ship/" + "PRC Jiangkai I Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Jiangkai II Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Jiangkai II Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Jiangkai II Class Ship/" + "PRC Jiangkai II Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Jiangwei II Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Jiangwei II Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Jiangwei II Class Ship/" + "PRC Jiangwei II Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Kuznetsov Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Kuznetsov Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Kuznetsov Class Ship/" + "PRC Kuznetsov Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Luhai Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Luhai Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Luhai Class Ship/" + "PRC Luhai Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Luhu Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Luhu Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Luhu Class Ship/" + "PRC Luhu Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Luyang I Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Luyang I Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Luyang I Class Ship/" + "PRC Luyang I Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Luyang II Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Luyang II Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Luyang II Class Ship" + "PRC Luyang II Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Luyang III Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Luyang III Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Luyang III Class Ship/" + "PRC Luyang III Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Luzhou Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Luzhou Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Luzhou Class Ship/" + "PRC Luzhou Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Renhai Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Renhai Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Renhai Class Ship/" + "PRC Renhai Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Shanghai III Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Shanghai III Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Shanghai III Class Ship/" + "PRC Shanghai III Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Sovremenny Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Sovremenny Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Sovremenny Class Ship/" + "PRC Sovremenny Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Wochi Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Wochi Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Wochi Class Ship/" + "PRC Wochi Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Wosao Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Wosao Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Wosao Class Ship/" + "PRC Wosao Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Wozang Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Wozang Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Wozang Class Ship/" + "PRC Wozang Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yubei Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yubei Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yubei Class Ship/" + "PRC Yubei Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yudeng Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yudeng Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yudeng Class Ship/" + "PRC Yudeng Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yuhai Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yuhai Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yuhai Class Ship/" + "PRC Yuhai Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yunshu Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yunshu Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yunshu Class Ship/" + "PRC Yunshu Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yushen Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yushen Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yushen Class Ship/" + "PRC Yushen Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yuting Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yuting Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yuting Class Ship/" + "PRC Yuting Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yuting II Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yuting II Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yuting II Class Ship/" + "PRC Yuting II Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yuting III Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yuting III Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yuting III Class Ship/" + "PRC Yuting III Class" + str(i) + ".jpg")

collection = "/content/drive/MyDrive/Practicum/Ships/PRC/Yuzhao Class Ship/"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("/content/drive/MyDrive/Practicum/Ships/PRC/Yuzhao Class Ship/" + filename, "/content/drive/MyDrive/Practicum/Ships/PRC/Yuzhao Class Ship/" + "PRC Yuzhao Class" + str(i) + ".jpg")


In [ ]:
import shutil
import os

source = '/content/drive/MyDrive/Practicum/Ships/IRIN/'
destination = '/content/drive/MyDrive/Practicum/Ships'

files = os.listdir(source)
 
for f in files:
    shutil.move(source + f, destination)

In [ ]:
source = '/content/drive/MyDrive/Practicum/Ships/PRC/'
destination = '/content/drive/MyDrive/Practicum/Ships'

files = os.listdir(source)
 
for f in files:
    shutil.move(source + f, destination)

In [ ]:
shutil.rmtree('/content/drive/MyDrive/Practicum/Ships/IRIN')
shutil.rmtree('/content/drive/MyDrive/Practicum/Ships/PRC')

In [ ]:
!pip install split_folders #Used to split the data set into training, testing, and validation data sets
import splitfolders

In [ ]:
splitfolders.ratio('/content/drive/MyDrive/Practicum/Ships', output="/content/drive/MyDrive/Practicum/Ships_final", seed=1337, ratio=(.7, 0.15,0.15))






Copying files: 0 files [00:00, ? files/s]



Copying files: 16 files [00:00, 153.68 files/s]



Copying files: 32 files [00:00, 118.57 files/s]



Copying files: 45 files [00:00, 117.23 files/s]



Copying files: 59 files [00:00, 122.81 files/s]



Copying files: 72 files [00:00, 121.76 files/s]



Copying files: 87 files [00:00, 126.07 files/s]



Copying files: 102 files [00:00, 131.05 files/s]



Copying files: 116 files [00:00, 128.16 files/s]



Copying files: 129 files [00:01, 126.27 files/s]



Copying files: 142 files [00:01, 125.38 files/s]



Copying files: 157 files [00:01, 129.72 files/s]



Copying files: 172 files [00:01, 134.54 files/s]



Copying files: 186 files [00:01, 134.23 files/s]



Copying files: 200 files [00:01, 131.60 files/s]



Copying files: 214 files [00:01, 129.67 files/s]



Copying files: 229 files [00:01, 134.08 files/s]



Copying files: 245 files [00:01, 134.96 files/s]



Copying files: 262 files [00:02, 134.94 files/s]



Copying files: 277 fi

In [ ]:
generator = ImageDataGenerator()
batches_consol   = generator.flow_from_directory('/content/drive/MyDrive/Practicum/Ships_final/train')

indices = batches_consol.class_indices

labels  = list(indices.keys())
print(labels)

Found 593 images belonging to 48 classes.
['.ipynb_checkpoints', 'Alvand Class Ship', 'BH7 Class Ship', 'Bandar Abbas Class Ship', 'Bayandor Class Ship', 'Delvar Class Ship', 'Fuyu Class Ship', 'Haiqing Class Ship', 'Hamzeh Class Ship', 'Hendijan Class Ship', 'Hengam Class Ship', 'Houbei Class Ship', 'Houjian Class Ship', 'Houxin Class Ship', 'Jiangdao Class Ship', 'Jianghu Class Ship', 'Jiangkai I Class Ship', 'Jiangkai II Class Ship', 'Jiangwei II Class Ship', 'Kaivan Class Ship', 'Kaman Class Ship', 'Kuznetsov Class Ship', 'Luhai Class Ship', 'Luhu Class Ship', 'Luyang I Class Ship', 'Luyang II Class Ship', 'Luyang III Class Ship', 'Luzhou Class Ship', 'Moudge Class Ship', 'Parvin Class Ship', 'Renhai Class Ship', 'SRN6 Class Ship', 'Shanghai III Class Ship', 'Sina Class Ship', 'Sovremenny Class Ship', 'Wochi Class Ship', 'Wosao Class Ship', 'Wozang Class Ship', 'Yubei Class Ship', 'Yudeng Class Ship', 'Yuhai Class Ship', 'Yunshu Class Ship', 'Yushen Class Ship', 'Yuting Class Ship'

In [ ]:
source = '/content/drive/MyDrive/Practicum/Ships_final/train/'
destination = '/content/drive/MyDrive/Practicum/Ships_final/train'

files = os.listdir(source)
 
for f in files:
    shutil.move(source + f, destination)

TypeError: ignored

In [ ]:
os.makedirs('/content/drive/MyDrive/Practicum/Ships_final/train/train_augmented')

In [ ]:
import numpy as np
import keras,glob,os
import cv2
from keras.preprocessing.image import ImageDataGenerator, array_to_img,img_to_array, load_img

img_path = '/content/drive/MyDrive/Practicum/Ships_final/train/'
outpath = '/content/drive/MyDrive/Practicum/Untitled folder/train_augmented'

filenames = glob.glob(img_path + "/**/*.jpg",recursive=True)


for img in filenames:

    if "DS_Store" in img: continue
    src_fname, ext = os.path.splitext(img) 

    datagen = ImageDataGenerator(        
            rotation_range=45,
            width_shift_range=0.2,  
            height_shift_range=0.2,    
            shear_range=0.2,        
            zoom_range=0.2,        
            horizontal_flip=True,         
            fill_mode='reflect')


    img = load_img(img)

    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    img_name = src_fname.split('/')[-1]
    new_dir = os.path.join(outpath, src_fname.split('/')[-1].rsplit('-', 50)[0])
    if not os.path.lexists(new_dir):
        os.mkdir(new_dir)
    #save_fname = os.path.join(new_dir, os.path.basename(img_name))
    save_fname = new_dir

    i = 0
    for batch in datagen.flow (x, batch_size=1, save_to_dir = save_fname, 
                               save_prefix = img_name, save_format='jpg'):
        i+=1
        if i>5:
            break

FileNotFoundError: ignored

In [ ]:
generator = ImageDataGenerator()
batches_consol   = generator.flow_from_directory('/content/drive/MyDrive/Practicum/Ships_final/train/train_augmented')

indices = batches_consol.class_indices

labels  = list(indices.keys())
print(labels)

Found 3555 images belonging to 593 classes.
['IRIN Alvand Class0', 'IRIN Alvand Class1', 'IRIN Alvand Class10', 'IRIN Alvand Class11', 'IRIN Alvand Class12', 'IRIN Alvand Class14', 'IRIN Alvand Class15', 'IRIN Alvand Class16', 'IRIN Alvand Class17', 'IRIN Alvand Class20', 'IRIN Alvand Class21', 'IRIN Alvand Class3', 'IRIN Alvand Class4', 'IRIN Alvand Class8', 'IRIN Alvand Class9', 'IRIN BH7 Class0', 'IRIN BH7 Class1', 'IRIN BH7 Class11', 'IRIN BH7 Class2', 'IRIN BH7 Class4', 'IRIN BH7 Class5', 'IRIN BH7 Class8', 'IRIN BH7 Class9', 'IRIN Bandar Abbas Class0', 'IRIN Bandar Abbas Class1', 'IRIN Bandar Abbas Class11', 'IRIN Bandar Abbas Class2', 'IRIN Bandar Abbas Class4', 'IRIN Bandar Abbas Class5', 'IRIN Bandar Abbas Class8', 'IRIN Bandar Abbas Class9', 'IRIN Bayandor Class0', 'IRIN Bayandor Class1', 'IRIN Bayandor Class10', 'IRIN Bayandor Class11', 'IRIN Bayandor Class12', 'IRIN Bayandor Class14', 'IRIN Bayandor Class15', 'IRIN Bayandor Class16', 'IRIN Bayandor Class17', 'IRIN Bayandor 

In [ ]:
RootDir1 = r'/content/drive/MyDrive/Practicum/Ships_final/train/train_augmented/'
TargetFolder = r'/content/drive/MyDrive/Practicum/Ships_final/train'
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):
        for name in files:
            if name.endswith('.jpg'):
                SourceFolder = os.path.join(root,name)
                shutil.move(SourceFolder, TargetFolder) #moves jpg to new folder

In [ ]:
import glob
import os
import shutil

In [ ]:
src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Alvand Class Ship'
# move file whose name starts with string '*'
pattern = src_folder + 'IRIN Alvand *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/BH7 Class Ship'
pattern = src_folder + 'IRIN BH7 *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Bandar Abbas Class Ship'
pattern = src_folder + 'IRIN Bandar *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Bayandor Class Ship'
pattern = src_folder + 'IRIN Bayandor *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Delvar Class Ship'
pattern = src_folder + 'IRIN Delvar *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Fuyu Class Ship'
pattern = src_folder + 'PRC Fuyu *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Haiqing Class Ship'
pattern = src_folder + 'PRC Haiqing *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Hamzeh Class Ship'
pattern = src_folder + 'IRIN Hamzeh *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Hendijan Class Ship'
pattern = src_folder + 'IRIN Hendijan *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Hengam Class Ship'
pattern = src_folder + 'IRIN Hengam *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Houbei Class Ship'
pattern = src_folder + 'PRC Houbei *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Houjian Class Ship'
pattern = src_folder + 'PRC Houjian *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Houxin Class Ship'
pattern = src_folder + 'PRC Houxin *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Jiangdao Class Ship'
pattern = src_folder + 'PRC Jiangdao *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Jianghu Class Ship'
pattern = src_folder + 'PRC Jianghu *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Jiangkai I Class Ship'
pattern = src_folder + 'PRC Jiangkai I *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Jiangkai II Class Ship'
pattern = src_folder + 'PRC Jiangkai II *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Jiangwei II Class Ship'
pattern = src_folder + 'PRC Jiangwei II *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Kaivan Class Ship'
pattern = src_folder + 'IRIN Kaivan *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Kaman Class Ship'
pattern = src_folder + 'IRIN Kaman *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Kuznetsov Class Ship'
pattern = src_folder + 'PRC Kuznetsov *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Luhai Class Ship'
pattern = src_folder + 'PRC Luhai *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Luhu Class Ship'
pattern = src_folder + 'PRC Luhu *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Luyang I Class Ship'
pattern = src_folder + 'PRC Luyang I *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Luyang II Class Ship'
pattern = src_folder + 'PRC Luyang II *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Luyang III Class Ship'
pattern = src_folder + 'PRC Luyang III *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Luzhou Class Ship'
pattern = src_folder + 'PRC Luzhou *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Moudge Class Ship'
pattern = src_folder + 'IRIN Moudge *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Parvin Class Ship'
pattern = src_folder + 'IRIN Parvin *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Renhai Class Ship'
pattern = src_folder + 'PRC Renhai *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/SRN6 Class Ship'
pattern = src_folder + 'IRIN SRN6 *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Shanghai III Class Ship'
pattern = src_folder + 'PRC Shanghai III *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Sina Class Ship'
pattern = src_folder + 'IRIN Sina *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Sovremenny Class Ship'
pattern = src_folder + 'PRC Sovremenny *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Wochi Class Ship'
pattern = src_folder + 'PRC Wochi *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Wosao Class Ship'
pattern = src_folder + 'PRC Wosao *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Wozang Class Ship'
pattern = src_folder + 'PRC Wozang *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yubei Class Ship'
pattern = src_folder + 'PRC Yubei *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yudeng Class Ship'
pattern = src_folder + 'PRC Yudeng *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yuhai Class Ship'
pattern = src_folder + 'PRC Yuhai *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yunshu Class Ship'
pattern = src_folder + 'PRC Yunshu *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yushen Class Ship'
pattern = src_folder + 'PRC Yushen *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yuting Class Ship'
pattern = src_folder + 'PRC Yuting *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yuting II Class Ship'
pattern = src_folder + 'PRC Yuting II *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yuting III Class Ship'
pattern = src_folder + 'PRC Yuting III *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

src_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/'
dst_folder = r'/content/drive/MyDrive/Practicum/Ships_final/train/Yuzhao Class Ship'
pattern = src_folder + 'PRC Yuzhao *'
for file in glob.iglob(pattern, recursive=True):
    # extract file name form file path
    #file_name = os.path.basename(file)
    shutil.move(file, dst_folder)
    print('Moved:', file)

In [3]:
generator = ImageDataGenerator()
batches_consol   = generator.flow_from_directory('/content/drive/MyDrive/Practicum/Ships_final/train')
batches_consol1   = generator.flow_from_directory('/content/drive/MyDrive/Practicum/Ships_final/test')
batches_consol2   = generator.flow_from_directory('/content/drive/MyDrive/Practicum/Ships_final/val')

indices = batches_consol.class_indices
indices1 = batches_consol1.class_indices
indices2 = batches_consol2.class_indices

labels  = list(indices.keys())
labels1  = list(indices1.keys())
labels2  = list(indices2.keys())

print(labels)
print(labels1)
print(labels2)

Found 4064 images belonging to 48 classes.
Found 171 images belonging to 47 classes.
Found 112 images belonging to 47 classes.
['.ipynb_checkpoints', 'Alvand Class Ship', 'BH7 Class Ship', 'Bandar Abbas Class Ship', 'Bayandor Class Ship', 'Delvar Class Ship', 'Fuyu Class Ship', 'Haiqing Class Ship', 'Hamzeh Class Ship', 'Hendijan Class Ship', 'Hengam Class Ship', 'Houbei Class Ship', 'Houjian Class Ship', 'Houxin Class Ship', 'Jiangdao Class Ship', 'Jianghu Class Ship', 'Jiangkai I Class Ship', 'Jiangkai II Class Ship', 'Jiangwei II Class Ship', 'Kaivan Class Ship', 'Kaman Class Ship', 'Kuznetsov Class Ship', 'Luhai Class Ship', 'Luhu Class Ship', 'Luyang I Class Ship', 'Luyang II Class Ship', 'Luyang III Class Ship', 'Luzhou Class Ship', 'Moudge Class Ship', 'Parvin Class Ship', 'Renhai Class Ship', 'SRN6 Class Ship', 'Shanghai III Class Ship', 'Sina Class Ship', 'Sovremenny Class Ship', 'Wochi Class Ship', 'Wosao Class Ship', 'Wozang Class Ship', 'Yubei Class Ship', 'Yudeng Class Shi